TRIGGERS

In [2]:
CREATE TRIGGER tr_Bichos on Bichos FOR INSERT as
begin
DECLARE @tipo VARCHAR(20), @especie VARCHAR(20)
    SELECT @tipo = bicho_tipo from inserted 
    SELECT @especie = bicho_especie from inserted
    if()

    BEGIN
    ROLLBACK tran
    RAISERROR('No es compatible el tipo con la especie del bicho agregado')
    end
end

GO

CREATE TRIGGER tr_PropietarioUpdate on Intercambios FOR INSERT as

   UPDATE Bichos set bicho_propietario = (SELECT usuario_uno from Intercambios WHERE bicho_dos = usuario_dos)
    
    
    
    
   /* DECLARE @usuario_uno INT, @usuario_dos INT, @bicho_uno INT, @bicho_dos INT, @bicho_propietario INT
    SELECT @bicho_propietario = bicho_propietario FROM Bichos
    SELECT @usuario_uno = usuario_uno FROM inserted
    SELECT @usuario_dos = usuario_dos FROM inserted
    SELECT @bicho_uno = bicho_uno FROM inserted
    SELECT @bicho_dos = bichco_dos FROM inserted
    */

Commands completed successfully.

Total execution time: 00:00:00.344